<a href="https://colab.research.google.com/github/muskang48/SurvCI/blob/main/surv_ci_example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
#Importing Libraries
import os
import pandas as pd
import numpy as np
import torch
import math
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import ParameterGrid

In [ ]:
SurvCI_dir = os.getcwd()
data_dir = SurvCI_dir + "/data/"
actg_dir = data_dir + "ACTG Data/"
Synthetic_SurvITE_dir = data_dir + "Synthetic_SurvITE/"


S-3 setting

In [ ]:
s3_train = np.load(Synthetic_SurvITE_dir + "train_s3_[0, 1]_1.npz")
s3_test = np.load(Synthetic_SurvITE_dir + "test_s3_[0, 1]_1.npz") 

train_list = [s3_train]
test_list = [s3_test]
names = ["s3_[0, 1]_1"]

S-4 setting

In [2]:
s4_train = np.load(Synthetic_SurvITE_dir + "train_s4_[0, 1]_1.npz")
s4_test = np.load(Synthetic_SurvITE_dir + "test_s4_[0, 1]_1.npz") 

train_list = [s4_train]
test_list = [s4_test]
names = ["s4_[0, 1]_1"]

S-1 setting

In [ ]:
s1_train = np.load(Synthetic_SurvITE_dir+"train_s1.npz")
s1_test = np.load(Synthetic_SurvITE_dir+"test_s1.npz")

train_list = [s1_train]
test_list = [s1_test]
names = ["s1"]

S-2 setting

In [ ]:
s2_train = np.load(Synthetic_SurvITE_dir+"train_s2.npz")
s2_test = np.load(Synthetic_SurvITE_dir+"test_s2.npz")

train_list = [s2_train]
test_list = [s2_test]
names = ["s2"]

ACTG_S3

In [2]:
actg_s3_train_gt = pd.read_csv(actg_dir+"s3/train/event_pairs.csv")
actg_s3_test_gt = pd.read_csv(actg_dir+"s3/test/event_pairs.csv")
actg_s3_val_gt = pd.read_csv(actg_dir+"s3/val/event_pairs.csv")
actg_s3_train_x = np.load(actg_dir+"s3/train/covariates.npy")
actg_s3_test_x = np.load(actg_dir+"s3/test/covariates.npy")
actg_s3_val_x = np.load(actg_dir+"s3/val/covariates.npy")
actg_s3_train_a = np.load(actg_dir+"s3/train/treatment.npy")
actg_s3_test_a = np.load(actg_dir+"s3/test/treatment.npy")
actg_s3_val_a = np.load(actg_dir+"s3/val/treatment.npy")

train_list = [[actg_s3_train_gt,actg_s3_train_x,actg_s3_train_a]]
test_list = [[actg_s3_test_gt,actg_s3_test_x,actg_s3_test_a]]
val_list = [[actg_s3_val_gt,actg_s3_val_x,actg_s3_val_a]]
names = ["actg_s3"]

ACTG_S4

In [2]:
actg_s4_train_gt = pd.read_csv(actg_dir+"s4/train/event_pairs.csv")
actg_s4_test_gt = pd.read_csv(actg_dir+"s4/test/event_pairs.csv")
actg_s4_val_gt = pd.read_csv(actg_dir+"s4/val/event_pairs.csv")
actg_s4_train_x = np.load(actg_dir+"s4/train/covariates.npy")
actg_s4_test_x = np.load(actg_dir+"s4/test/covariates.npy")
actg_s4_val_x = np.load(actg_dir+"s4/val/covariates.npy")
actg_s4_train_a = np.load(actg_dir+"s4/train/treatment.npy")
actg_s4_test_a = np.load(actg_dir+"s4/test/treatment.npy")
actg_s4_val_a = np.load(actg_dir+"s4/val/treatment.npy")

train_list = [[actg_s4_train_gt,actg_s4_train_x,actg_s4_train_a]]
test_list = [[actg_s4_test_gt,actg_s4_test_x,actg_s4_test_a]]
val_list = [[actg_s4_val_gt,actg_s4_val_x,actg_s4_val_a]]
names = ["actg_s4"]



# SurvCI Model 

In [ ]:
%cd "{SurvCI_dir}/survci"

In [4]:
#Importing Model
from survci.survci_api import survciBase

ACTG data Training Loop

In [ ]:
import pdb
#k,p_alpha,p_beta,
p_alpha=[0.5,0.6,0.7]
p_beta=[1e-4,2e-4,3e-4]
p_lamda = [2e-1]
p_gamma = [1e-1]
k = [3]

for i,j,k,l in zip(train_list,test_list,val_list,names):
    x_train = SimpleImputer(missing_values=np.nan, strategy='mean').fit_transform(i[1])
    x_test = SimpleImputer(missing_values=np.nan, strategy='mean').fit_transform(j[1])
    x_val = SimpleImputer(missing_values=np.nan, strategy='mean').fit_transform(k[1])
    x_train = StandardScaler().fit_transform(x_train)
    x_test = StandardScaler().fit_transform(x_test)
    x_val = StandardScaler().fit_transform(x_val)
    # train
    tf_train = i[0]['y_f']
    tcf_train = i[0]['y_cf']
    ef_train = i[0]['e_f']
    ecf_train = i[0]['e_cf']
    W_train = i[2]
    # test
    tf_test = j[0]['y_f']
    tcf_test = j[0]['y_cf']
    ef_test = j[0]['e_f']
    ecf_test = j[0]['e_cf']
    W_test = j[2]
    # val
    tf_val = k[0]['y_f']
    tcf_val = k[0]['y_cf']
    ef_val = k[0]['e_f']
    ecf_val = k[0]['e_cf']
    W_val = k[2]
    param_grid = {'k' : [3],    #Number of Parametric Distibutions
                'distribution' : ['LogNormal'],   
                'learning_rate' : [3e-4],
                'layers' : [[100,100]],
                'discount': [1],     #(Scaling ELBO_censored Loss)
                'imb_func': ['mmd2_lin'],  
                'p_alpha': p_alpha,       #Hyperparameter for Scaling IPM-Imbalance
                'p_beta': p_beta,   #Hyperparameter for Scaling MSE Loss
                'p_gamma':p_gamma, #Hyperparameter for Scaling ELBO Loss
                'p_lamda': p_lamda   #Hyperparameter for scaling Regularization Loss
                }
    params = ParameterGrid(param_grid)

    models = []
    ci=[]
    for param in params:
        model = survciBase(k = param['k'],
                                    distribution = param['distribution'],
                                    layers = param['layers'],
                                    discount=param['discount'],
                                    imb_func=param['imb_func'],p_alpha=param['p_alpha'],p_beta = param['p_beta'],p_lambda=param["p_lamda"],name=l)
        # The fit method is called to train the model
        model.fit(x_train, np.array(tf_train), np.array(ef_train),W_train, iters = 200,learning_rate = param['learning_rate'],batch_size=3500,val_data=(x_val, np.array(tf_val), np.array(ef_val),W_val)) 

S1 Training Loop

In [ ]:
import pdb
#k,p_alpha,p_beta,
p_alpha=[0.6]
p_beta=[3e-4]
p_lamda = [2e-1]
p_gamma = [1e-1]

for i,j,k in zip(train_list,test_list,names):
    x_train = i["x"]
    x_train = SimpleImputer(missing_values=np.nan, strategy='mean').fit_transform(x_train)
    x_train = StandardScaler().fit_transform(x_train)
    tf_train = i['t']
    W_train = i["a"]
    ef_train = np.array([1 for i in range(len(W_train))])

    x_test = j["x"]
    x_test = SimpleImputer(missing_values=np.nan, strategy='mean').fit_transform(x_test)
    x_test = StandardScaler().fit_transform(x_test)
    tf_test = j["t"]
    W_test = j["a"]
    ef_test = np.array([1 for i in range(len(W_test))])

    n = len(x_train)
    tr_size = int(n*0.70)
    vl_size = int(n*0.30)

    x_train,x_val = x_train[:tr_size], x_train[tr_size:tr_size+vl_size]
    tf_train,tf_val = tf_train[:tr_size], tf_train[tr_size:tr_size+vl_size]
    ef_train,ef_val = ef_train[:tr_size], ef_train[tr_size:tr_size+vl_size]
    W_train,W_val = W_train[:tr_size], W_train[tr_size:tr_size+vl_size]
    rand1 = np.random.randint(0,len(W_val),20)
    rand2 = np.random.randint(0,len(W_val),20)
    rand3 = np.random.randint(0,len(W_val),20)
    rand4 = np.random.randint(0,len(W_val),20)
    for i,j,k,l in zip(rand1,rand2,rand3,rand4):
        ef_train[i] = 0
        W_train[j] = 1
        ef_val[k] = 0
        ef_val[l] = 0
    param_grid = {'k' : [3],    #Number of Parametric Distibutions
                'distribution' : ['LogNormal'],   
                'learning_rate' : [3e-4],
                'layers' : [[100,100]],
                'discount': [1],     #(Scaling ELBO_censored Loss)
                'imb_func': ['mmd2_lin'],  
                'p_alpha': p_alpha,       #Hyperparameter for Scaling IPM-Imbalance
                'p_beta': p_beta,   #Hyperparameter for Scaling MSE Loss
                'p_gamma':p_gamma, #Hyperparameter for Scaling ELBO Loss
                'p_lamda': p_lamda   #Hyperparameter for scaling Regularization Loss
                }
    params = ParameterGrid(param_grid)
    for param in params:
        model = survciBase(k = param['k'],
                                    distribution = param['distribution'],
                                    layers = param['layers'],
                                    discount=param['discount'],
                                    imb_func=param['imb_func'],p_alpha=param['p_alpha'],p_beta = param['p_beta'],p_lambda=param["p_lamda"],name=k)
        # The fit method is called to train the model
        model.fit(x_train, tf_train, ef_train,W_train, iters = 200,learning_rate = param['learning_rate'],batch_size=3500,val_data=(x_val, tf_val, ef_val,W_val))

S2 Training Loop

In [ ]:
import pdb
p_alpha=[0.6]
p_beta=[3e-4]
p_lamda = [2e-1]
p_gamma = [1e-1]

for i,j,k in zip(train_list,test_list,names):
    x_train = i["x"]
    x_train = SimpleImputer(missing_values=np.nan, strategy='mean').fit_transform(x_train)
    x_train = StandardScaler().fit_transform(x_train)
    tf_train = i['t']
    W_train = i["a"]
    ef_train = i["e"]

    x_test = j["x"]
    x_test = SimpleImputer(missing_values=np.nan, strategy='mean').fit_transform(x_test)
    x_test = StandardScaler().fit_transform(x_test)
    tf_test = j["t"]
    W_test = j["a"]
    ef_test = j["e"]

    n = len(x_train)
    tr_size = int(n*0.70)
    vl_size = int(n*0.30)

    x_train,x_val = x_train[:tr_size], x_train[tr_size:tr_size+vl_size]
    tf_train,tf_val = tf_train[:tr_size], tf_train[tr_size:tr_size+vl_size]
    ef_train,ef_val = ef_train[:tr_size], ef_train[tr_size:tr_size+vl_size]
    W_train,W_val = W_train[:tr_size], W_train[tr_size:tr_size+vl_size]
    rand1 = np.random.randint(0,len(W_val),20)
    rand2 = np.random.randint(0,len(W_val),20)
    for i,j in zip(rand1,rand2):
        ef_train[i] = 0
        W_train[j] = 1

    param_grid = {'k' : [3],    #Number of Parametric Distibutions
                'distribution' : ['LogNormal'],   
                'learning_rate' : [3e-4],
                'layers' : [[100,100]],
                'discount': [1],     #(Scaling ELBO_censored Loss)
                'imb_func': ['mmd2_lin'],  
                'p_alpha': p_alpha,       #Hyperparameter for Scaling IPM-Imbalance
                'p_beta': p_beta,   #Hyperparameter for Scaling MSE Loss
                'p_gamma':p_gamma, #Hyperparameter for Scaling ELBO Loss
                'p_lamda': p_lamda   #Hyperparameter for scaling Regularization Loss
                }
    params = ParameterGrid(param_grid)
    for param in params:
        model = survciBase(k = param['k'],
                                    distribution = param['distribution'],
                                    layers = param['layers'],
                                    discount=param['discount'],
                                    imb_func=param['imb_func'],p_alpha=param['p_alpha'],p_beta = param['p_beta'],p_lambda=param["p_lamda"],name=k)
        # The fit method is called to train the model
        model.fit(x_train, tf_train, ef_train,W_train, iters = 200,learning_rate = param['learning_rate'],batch_size=3500,val_data=(x_val, tf_val, ef_val,W_val))

S3 Training Loop


In [ ]:
import pdb
p_alpha=[0.6]
p_beta=[3e-4]
p_lamda = [2e-1]
p_gamma = [1e-1]

for i,j,k in zip(train_list,test_list,names):
    x_train = i["x"]
    x_train = SimpleImputer(missing_values=np.nan, strategy='mean').fit_transform(x_train)
    x_train = StandardScaler().fit_transform(x_train)
    # tf_train = i['t']
    tf_train = i['t_f']
    tcf_train = i["t_cf"]
    W_train = i["a"]
    ef_train = np.array([1 for i in range(len(W_train))])

    x_test = j["x"]
    x_test = SimpleImputer(missing_values=np.nan, strategy='mean').fit_transform(x_test)
    x_test = StandardScaler().fit_transform(x_test)
    tf_test = j['t_f']
    tcf_test = j["t_cf"]
    W_test = j["a"]
    ef_test = np.array([1 for i in range(len(W_train))])

    n = len(x_train)
    tr_size = int(n*0.70)
    vl_size = int(n*0.30)

    x_train,x_val = x_train[:tr_size], x_train[tr_size:tr_size+vl_size]
    tf_train,tf_val = tf_train[:tr_size], tf_train[tr_size:tr_size+vl_size]
    ef_train,ef_val = ef_train[:tr_size], ef_train[tr_size:tr_size+vl_size]
    tcf_train,tcf_val = tcf_train[:tr_size], tcf_train[tr_size:tr_size+vl_size]
    W_train,W_val = W_train[:tr_size], W_train[tr_size:tr_size+vl_size]
    rand1 = np.random.randint(0,len(W_val),20)
    rand2 = np.random.randint(0,len(W_val),20)
    for i,j in zip(rand1,rand2):
        ef_train[i] = 0
        W_train[j] = 1
    param_grid = {'k' : [3],    #Number of Parametric Distibutions
                'distribution' : ['LogNormal'],   
                'learning_rate' : [3e-4],
                'layers' : [[100,100]],
                'discount': [1],     #(Scaling ELBO_censored Loss)
                'imb_func': ['mmd2_lin'],  
                'p_alpha': p_alpha,       #Hyperparameter for Scaling IPM-Imbalance
                'p_beta': p_beta,   #Hyperparameter for Scaling MSE Loss
                'p_gamma':p_gamma, #Hyperparameter for Scaling ELBO Loss
                'p_lamda': p_lamda   #Hyperparameter for scaling Regularization Loss
                }
    params = ParameterGrid(param_grid)
    for param in params:
        model = survciBase(k = param['k'],
                                    distribution = param['distribution'],
                                    layers = param['layers'],
                                    discount=param['discount'],
                                    imb_func=param['imb_func'],p_alpha=param['p_alpha'],p_beta = param['p_beta'],p_lambda=param["p_lamda"],name=k)
        # The fit method is called to train the model
        model.fit(x_train, tf_train, ef_train,W_train, iters = 200,learning_rate = param['learning_rate'],batch_size=3500,val_data=(x_val, tf_val, ef_val,W_val))

S4 Training Loop

In [ ]:
import pdb
p_alpha=[0.6]
p_beta=[3e-4]
p_lamda = [2e-1]
p_gamma = [1e-1]

for i,j,k in zip(train_list,test_list,names):
    x_train = i["x"]
    x_train = SimpleImputer(missing_values=np.nan, strategy='mean').fit_transform(x_train)
    x_train = StandardScaler().fit_transform(x_train)
    tf_train = i['t_f']
    tcf_train = i["t_cf"]
    W_train = i["a"]
    ef_train = i["e_f"]
    ef_train = np.array([1 for i in range(len(W_train))])

    x_test = j["x"]
    x_test = SimpleImputer(missing_values=np.nan, strategy='mean').fit_transform(x_test)
    x_test = StandardScaler().fit_transform(x_test)
    tf_test = j['t_f']
    tcf_test = j["t_cf"]
    W_test = j["a"]
    ef_test = j["e_f"]
    ef_test = np.array([1 for i in range(len(W_test))])

    n = len(x_train)
    tr_size = int(n*0.70)
    vl_size = int(n*0.30)

    x_train,x_val = x_train[:tr_size], x_train[tr_size:tr_size+vl_size]
    tf_train,tf_val = tf_train[:tr_size], tf_train[tr_size:tr_size+vl_size]
    ef_train,ef_val = ef_train[:tr_size], ef_train[tr_size:tr_size+vl_size]
    tcf_train,tcf_val = tcf_train[:tr_size], tcf_train[tr_size:tr_size+vl_size]
    W_train,W_val = W_train[:tr_size], W_train[tr_size:tr_size+vl_size]
    param_grid = {'k' : [3],    #Number of Parametric Distibutions
                'distribution' : ['LogNormal'],   
                'learning_rate' : [3e-4],
                'layers' : [[100,100]],
                'discount': [1],     #(Scaling ELBO_censored Loss)
                'imb_func': ['mmd2_lin'],  
                'p_alpha': p_alpha,       #Hyperparameter for Scaling IPM-Imbalance
                'p_beta': p_beta,   #Hyperparameter for Scaling MSE Loss
                'p_gamma':p_gamma, #Hyperparameter for Scaling ELBO Loss
                'p_lamda': p_lamda   #Hyperparameter for scaling Regularization Loss
                }
    params = ParameterGrid(param_grid)
    print(x_train.shape)
    for param in params:
        model = survciBase(k = param['k'],
                                    distribution = param['distribution'],
                                    layers = param['layers'],
                                    discount=param['discount'],
                                    imb_func=param['imb_func'],p_alpha=param['p_alpha'],p_beta = param['p_beta'],p_lambda=param["p_lamda"],name=k)
        # The fit method is called to train the model
        model.fit(x_train, tf_train, ef_train,W_train, iters = 200,learning_rate = param['learning_rate'],batch_size=3500,val_data=(x_val, tf_val, ef_val,W_val))